In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install internetarchive
import internetarchive
import multiprocessing

# Replace with your collection/item ID
item_id = "EPI63"

# Fetch metadata
item = internetarchive.get_item(item_id)
files = [file["name"] for file in item.files]
files_mp3 = [file for file in files if file.endswith(".mp3")]
len_files_mp3 = len(files_mp3)

In [ ]:
selected_files = files_mp3

In [ ]:
# Display file list
# print(file_mp3) for file_mp3 in files_mp3
print(len_files_mp3)
print(files_mp3)

In [ ]:
# Download selected files
# for file in files_mp3:
#     internetarchive.download(item_id, files=[file], destdir="./EPI63")

In [ ]:
# Function to download a single file
def download_file(file_name):
    try:
        internetarchive.download(item_id, files=[file_name], destdir="./EPI63")
        print(f"Downloaded: {file_name}")
    except Exception as e:
        print(f"Failed: {file_name} - {e}")

In [ ]:
# Function to run downloads in batches of 20
def download_in_batches(file_list, batch_size=20):
    for i in range(0, len(file_list), batch_size):
        batch = file_list[i:i + batch_size]
        with multiprocessing.Pool(processes=batch_size) as pool:
            pool.map(download_file, batch)

In [ ]:
# Start downloading in batches of 20
download_in_batches(selected_files, batch_size=20)

In [ ]:
!ls

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# !zip -r /kaggle/working/EPI63.zip ./EPI63

In [ ]:
import os
os.makedirs("/kaggle/working/Ramayan", exist_ok=True)
base_dir = "/kaggle/working/EPI63/EPI63/"
episode_list = [file for file in os.listdir(base_dir)]
print(episode_list)
print(len(episode_list))

In [ ]:
# Rename to episode name only
for episode in episode_list:
    episode_name = episode.split(".")[0]
    episode_name = episode_name.replace("EPI-", "")
    print(episode_name,end="->")
    episode_name_3digit = episode_name.zfill(3)
    print(episode_name_3digit)
    os.rename(f"{base_dir}{episode}", f"{base_dir}{episode_name_3digit}.mp3")

In [ ]:
# rename ram siya ram .mp3 to 000_Ram_Siya_Ram.mp3
os.rename(f"{base_dir}Ram Siya Ram .mp3", f"{base_dir}000_Ram_Siya_Ram.mp3")

In [ ]:
renamed_episode_list = [file for file in os.listdir(base_dir)]
print(renamed_episode_list)
print(len(renamed_episode_list))

In [ ]:
sorted_renamed_episode_list = sorted(renamed_episode_list)
print(sorted_renamed_episode_list)
print(len(sorted_renamed_episode_list))

In [ ]:
# from 001.mp3 combine audio files in batches of 10 and rename them to 001_010.mp3 etc
!pip install pydub
!apt-get install ffmpeg
import os
from pydub import AudioSegment

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/working/Ramayan'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import os
import subprocess
import multiprocessing

audio_dir = base_dir  # Change to your directory
audio_files = sorted_renamed_episode_list[1:]
output_dir = "/kaggle/working/Ramayan/"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def merge_audio_ffmpeg(batch):
    start_num = batch[0].split('.')[0]
    end_num = batch[-1].split('.')[0]
    output_file = f"{output_dir}{start_num}_{end_num}.mp3"
    output_path = os.path.join(audio_dir, output_file)

    # Create input file list
    input_list_file = os.path.join(audio_dir, f"input_list_{start_num}.txt")
    with open(input_list_file, "w") as f:
        for file in batch:
            f.write(f"file '{os.path.join(audio_dir, file)}'\n")

    # Run FFmpeg in parallel
    subprocess.run(["ffmpeg", "-y", "-f", "concat", "-safe", "0", "-i", input_list_file, "-c", "copy", output_path], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    print(f"✅ Merged: {output_file}")

def merge_audio_batches_parallel(files, batch_size=10):
    batches = [files[i:i + batch_size] for i in range(0, len(files), batch_size)]

    with multiprocessing.Pool(processes=4) as pool:  # Adjust number of parallel processes
        pool.map(merge_audio_ffmpeg, batches)

# Run parallel merging
merge_audio_batches_parallel(audio_files, batch_size=10)


In [ ]:
# copy 000 to new folder
!cp /kaggle/working/EPI63/EPI63/000_Ram_Siya_Ram.mp3 /kaggle/working/Ramayan/
print("Succesfully copied 000_Ram_Siya_Ram.mp3")

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/working/Ramayan'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!mkdir /kaggle/working/Final/
!zip -r /kaggle/working/Final/Ramayan_Combined.zip /kaggle/working/Ramayan/

In [ ]:
print("Done")

In [ ]:
from IPython.display import FileLink

# Path to the file you want to link to
file_path = '/kaggle/working/Final/Ramayan_Combined.zip'

# Create a clickable link
display(FileLink(file_path))


In [ ]:
from IPython.display import HTML

# Generate a downloadable link for the file in Kaggle notebook's Output directory
def create_download_link(file_path):
    # Replace spaces with %20 for URL encoding
    file_path = file_path.replace(" ", "%20")
    return HTML(f'<a href="{file_path}" download>Click here to download the file</a>')

# File path of your .zip file
file_path = '/kaggle/working/output.zip'

# Display the download link in the notebook
create_download_link(file_path)